In [ ]:
from my_import import *

#uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"
#uri = "192.168.99.10:5555"
#uri = "mirror-ecat-srv.local:5555"

io = ZmsgIO(uri)
io.debug = False

def reply_cmd(cmd):
    reply = io.doit(cmd)
    yaml_msg = yaml.safe_load(reply['msg'])
    return yaml_msg

status_cmds = {
    'FAN_ON':           0x0026,
    'FAN_OFF':			0x0062,
    'LED_ON':           0x0019,
    'LED_OFF':          0x0091,
    'RELEASE_BRAKE':    0x00BD,
    'ENGAGE_BRAKE':     0x00DB,
}
reply_cmds = {
    'CMD_DONE':         0x7800,
    'CMD_WORKING':		0xD000,
    'CMD_ERROR':		0xAA00,
    'CMD_NOCOND':       0xBB00,
}

def ctrl_status_cmd(ctrl_cmd: int, bid: int):
    msg = reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'ctrl_status_cmd': ctrl_cmd}).set_bid(bid))
    msg = reply_cmd(SdoCmd(rd_sdo=['ctrl_status_cmd_ack'],wr_sdo={}).set_bid(bid))
    print(hex(msg['ctrl_status_cmd_ack']))
    return msg['ctrl_status_cmd_ack'] == ctrl_cmd + reply_cmds['CMD_DONE']

def ctrl_status_cmd_str(ctrl_cmd: str, bid: int):
    return ctrl_status_cmd(status_cmds[ctrl_cmd], bid) 

def sdo_filter(snames:list, sdos:dict):
    return {key:sdos[key] for key in snames}

In [ ]:
scan_ids = reply_cmd(master_cmd_get_slave_descr).keys()
ids=list(scan_ids)
print(ids)
#assert(iD in ids)

In [ ]:
iD = 1
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))

In [ ]:
sf =sdo_filter(['m3_fw_ver','c28_fw_ver','hw_config_low','hw_config_high'],sdo_dict)
print(sf)
hex( (sf['hw_config_high']<<16) + sf['hw_config_low'] )

In [ ]:
# release all ids brakes using aux_pdo command
brk_rel = unpack('f',pack('i',1))[0]
for _id in ids :
    reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_cmd_control': brk_rel}).set_bid(_id))
    time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_cmd_status'],wr_sdo={}).set_bid(_id))['master_cmd_status']
    print(_id, ret)

In [ ]:
# engage all ids brakes using aux_pdo command
brk_eng = unpack('f',pack('i',2))[0]
for _id in ids :
    reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_cmd_control': brk_eng}).set_bid(_id))
    time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_cmd_status'],wr_sdo={}).set_bid(_id))['master_cmd_status']
    print(_id, ret)

In [ ]:
sdo = ['torque','motor_pos','link_pos','Min_pos','Max_pos','Max_cur']
sdo = ['link_pos','fault','linkEncBadReadPPM','absLinkEncErr']
for i in range(2):
    G = io.doit4ids([iD],SdoCmd(rd_sdo=sdo,wr_sdo={}))
    #[g for g in G ]
    for g in G :
        print(g['msg'])
    time.sleep(0.2)

In [ ]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0x3B))

In [ ]:
io.doit(ctrl_cmd_stop.set_bid(iD))

In [ ]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4))

In [ ]:
io.debug=0
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 250
gains['kd'] = 5
#gains['ki'] = 0
gains['kkp'] = 2
gains['kkd'] = 0.001
gains['kki'] = 0.08
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(*gains.values()))
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',2000,10,2,0.001,0))
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',0,0,5,0.001,0.08))

In [ ]:
io.doit(ctrl_cmd_stop.set_bid(iD))

In [ ]:
for _id in ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', _id)

In [ ]:
for _id in ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', _id)

In [ ]:
ctrl_status_cmd_str('FAN_ON', iD)

In [ ]:
ctrl_status_cmd_str('FAN_OFF', iD)

In [ ]:
io.doit(SdoCmd(rd_sdo=[],wr_sdo={'Max_cur': 10}).set_bid(iD))

In [ ]:
io.doit(flash_cmd_save_flash.set_bid(iD))

In [ ]:
iD


In [ ]:
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(16))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(16))